Scrap websites mentioned in file to collect food categories. It saves scrapped data after each page. 

In [23]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from bs4 import Comment
import time
from tqdm import tqdm

## Read file

You have to specified number of rows to read from a file - check first how many rows are there in the scv file.
Without 'nrows' parameter an exception occures.

In [37]:
pathToRead = 'C:/Users/Patrycja/Desktop/Uni Koblenz/CSS/project brudnopis/food_data_excerpt/food_data_excerpt_0v.csv'#'food_data_excerpt/food_data_excerpt.csv'
nrows = 336936

In [38]:
df = pd.read_csv(pathToRead, sep=',', encoding='utf8', nrows=10)#, engine='python'

In [39]:
df

,country,country_region,zip,city,date,target_link,origin_link,host_type,recipe_id
0,DE,7,47169,Duisburg,2013-02-28 23:59:59,/rezept/443097/Abendbrot-UEberbackenes-Broetch...,http://www.kochbar.de/kochen/Auflaeufe-zuberei...,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:2.0.1) ...,443097
1,DE,9,66564,Ottweiler,2013-03-01 00:00:01,/rezept/108268/Coq-au-vin.html,http://www.kochbar.de/rezepte/coq-au-vin.html,Mozilla/5.0 (Windows NT 5.1; rv:19.0) Gecko/20...,108268
2,DE,7,-1,Hattingen,2013-03-01 00:00:02,/rezept/231469/Kuchen-Blechkuchen-Kaesekuchen-...,-,Mozilla/5.0 (iPad; CPU OS 6_0_1 like Mac OS X)...,231469
3,DE,5,-1,Sulzbach,2013-03-01 00:00:04,/rezept/410839/Bier-Wackler-Dessert.html,http://www.google.de/url?sa=t&rct=j&q=wackler%...,Mozilla/5.0 (Windows NT 6.2; WOW64) AppleWebKi...,410839
4,DE,11,-1,Calau,2013-03-01 00:00:06,/rezept/392545/Black-Bottom-Cupcake.html,http://www.kochbar.de/rezepte/cupcakes.html?of...,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:19.0) G...,392545
5,DE,6,-1,Rhade,2013-03-01 00:00:06,/rezept/233705/Nudelpfanne-mit-Koettbullar.html,http://www.kochbar.de/rezepte/alle-rezepte.htm...,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,233705
6,DE,5,65931,Frankfurt_Am_Main,2013-03-01 00:00:07,/rezept/158488/Bechamelsosse-herzhaft.html,http://www.kochbar.de/rezepte/bechamelso%C3%9F...,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,158488
7,DE,7,47574,Goch,2013-03-01 00:00:07,/rezept/285300/Himmel-auf-Erden.html,http://www.kochbar.de/rezepte/sauerkrautauflau...,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:19.0) G...,285300
8,DE,9,66111,Saarbrücken,2013-03-01 00:00:08,/rezept/179340/Lachstartar.html,http://www.kochbar.de/rezepte/lachstartar.html,Mozilla/5.0 (Windows NT 5.1; rv:8.0.1) Gecko/2...,179340
9,DE,7,58644,Iserlohn,2013-03-01 00:00:09,/rezept/1999/Massaman-Curry-Paste.html,http://www.kochbar.de/kochrezepte/schmeckt-nic...,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,1999


## Collect data

In [5]:
def find_comment(soup, wanted_comment='REZEPT-KATEGORIEN'):
    found = False

    comments = soup.find_all(string=lambda text:isinstance(text, Comment))
    i = 0
    while not found and i < len(comments):
        node = comments[i]
        if node.strip() == wanted_comment:
            #print('comment found')
            found = True

        i+=1
    return node

In [6]:
def find_categories(soup, wanted_class = 'rtli-pb-small'):
    data = []
    node = find_comment(soup)
    startOfItems, endOfItems = False, False
    
    while not endOfItems and node.find_next_sibling():
        node = node.find_next_sibling()

        if node['class'][0] == wanted_class:
            startOfItems = True

            if node.findChild().name == 'a':
                data.append(node.a.text)
            elif node.findChild().name == 'span':
                data.append(node.span.text)

        elif startOfItems:
            endOfItems = True
    
    return data

In [7]:
domain = 'http://www.kochbar.de'

Remember to specify with rows to process. If 'all_rows' is set to True, 'start_index' and 'end_index' is not taken into account.

In [101]:
start_index, end_index = 110, 120
all_rows = False

In [180]:
def listToString(listToConvert):
    result = '['
    for el in listToConvert:
        result += el + ','
    
    result = result[:-1]
    return result +']'

In [308]:
def improveText(text):
    text = text.replace('ê','e')
    text = text.replace('Ê','E')
    
    text = text.replace('ö','oe')
    text = text.replace('Ö','Oe')
    text = text.replace('ä','ae')
    text = text.replace('Ä','Ae')
    text = text.replace('ü','ue')
    text = text.replace('Ü','Ue')
    text = text.replace('ß','ss')

    for i in range(len(text)):
        nr = ord(text[i])
        if (nr >= 97 and nr <= 122) or (nr >=65 and nr <=90) or nr in [32, 34, 38, 39, 44, 45, 47, 91, 92, 93]:
            pass
        else:
            print(text[i])
            text = text.replace(text[i], '?')
                
    return text

In [355]:
pathToSave = 'scrapped_data.csv'

In [351]:
start = time.time()

error_num = 0
df_new = pd.DataFrame(columns=['date', 'recipe_id', 'country', 'region', 'city', 'categories'])
f= open(pathToSave,'w')
df_new.to_csv(f, index=None, header=True, encoding='utf-8', sep=',')
f.close()

nrows = 336936
df = pd.read_csv(pathToRead, sep=',', encoding='utf-8', nrows=nrows)#, engine='python'

f= open(pathToSave, 'a')

if not all_rows:
    df = df[start_index:end_index]
else:
    start_index = 0

with tqdm(total=nrows) as pbar:
    for i in range(len(df)): 
        #print(domain + url)
        url = df['target_link'][start_index + i]
        response = requests.get(domain + url)

        if response.url != 'https://www.kochbar.de/rezepte/' and response.status_code == 200:
            #print('Request status ok!')
            soup = BeautifulSoup(response.content.decode('utf-8','ignore'), "lxml")
            categories = find_categories(soup)
            #print('categories:', categories, '\n')
            #data.append(categories)

            s = pd.DataFrame({'d': str(df['date'][start_index + i]), 'id': df['recipe_id'][start_index + i], 'country':df['country'][start_index + i], 'region':df['country_region'][start_index + i], 'city': improveText(df['city'][start_index + i]),'categories': improveText(str(categories))}, index=[0])
        elif response.status_code == 200:
            #print('This webpage is probably no longer available!')
            #data.append(None)
            s = pd.DataFrame({'d': df['date'][start_index + i], 'id': df['recipe_id'][start_index + i], 'country':df['country'][start_index + i], 'region':df['country_region'][start_index + i], 'city': improveText(df['city'][start_index + i]), 'categories': str([])}, index=[0])

        else:
            #print('Not correct request status: ', response.status_code)
            s = pd.DataFrame({'d': df['date'][start_index + i], 'id': df['recipe_id'][start_index + i], 'country':df['country'][start_index + i], 'region':df['country_region'][start_index + i], 'city': improveText(df['city'][start_index + i]), 'categories': str([])}, index=[0])
        try:
            s.to_csv(f, index=None, header=False, encoding='utf-8', sep=',')
        except UnicodeEncodeError:
            print('UnicodeEncodeError')
            print(s)
            error_num += 1

        pbar.update(1)

f.close() 
print(time.time() - start)
print('errors:', error_num)

  0%|          | 10/336936 [00:03<33:38:31,  2.78it/s]


6.656256437301636
errors: 0


In [302]:
f.close() 

## Check saved data

In [ ]:
df1=pd.read_csv(pathToSave)
df1

In [358]:
df1['categories'][1][2:-2].replace("'","").replace('"','').split(',')

['Europa',
 ' Fisch',
 ' Mittagstisch',
 ' Abendessen',
 ' Date',
 ' Fruehling',
 ' Sommer',
 ' Herbst',
 ' Winter',
 ' Hauptspeise',
 ' Gesund und Diaet',
 ' preiswerte',
 ' schnell & einfach',
 ' Italien',
 ' fettarm',
 ' glutenfrei',
 ' ohne Weizen']